In [1]:
# This file costructs surrogate models for the input datasets
import numpy as np   
import pandas as pd
import os
import shutil
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
import time
import warnings

# Torch specific module imports
import torch
import gpytorch 
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torch.nn import functional as F

# botorch specific modules
from botorch.fit import fit_gpytorch_model
from botorch.models.gpytorch import GPyTorchModel
from botorch import fit_gpytorch_mll
from botorch.acquisition.monte_carlo import (
    qExpectedImprovement,
    qNoisyExpectedImprovement,
)
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.acquisition import UpperConfidenceBound, ExpectedImprovement

# Plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Tick parameters
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1

plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['legend.fontsize'] = 15

# User defined python classes and files
import sys
sys.path.insert(0, '../scripts/')

import input_class 
import code_verification as verification
import surrogate_model_inputs as model_input
import utils_dataset as utilsd
import surrogate_models


/Users/maitreyeesharma/opt/anaconda3/envs/torch/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/maitreyeesharma/opt/anaconda3/envs/torch/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: /Users/maitreyeesharma/opt/anaconda3/envs/torch/lib/python3.11/site-packages/torchvision/image.so
  Expected in: /Users/maitreyeesharma/opt/anaconda3/envs/torch/lib/python3.11/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Using cpu device


In [2]:
random_seed = 1
# Reading the input test datafile
with open(model_input.run_folder+'inputs_testing.json', "r") as f:
    input_dict = json.load(f)

input_type = input_dict['InputType']
input_path = input_dict['InputPath']
input_file = input_dict['InputFile']
add_target_noise = input_dict['AddTargetNoise']

input = input_class.inputs(input_type=input_type,
                           input_path=input_path,
                           input_file=input_file,
                           add_target_noise=add_target_noise,
                           composition_MPEA = True)

XX, YY, descriptors = input.read_inputs(model_input.verbose)

# Reading the BO output files
if model_input.GP_0_BO:
    best_observed_df_gp0 = pd.read_csv(model_input.output_folder+'gp0_best.csv')
    newy_observed_df_gp0 = pd.read_csv(model_input.output_folder+'gp0_newTarget.csv')
    newy_var_observed_df_gp0 = pd.read_csv(model_input.output_folder+'gp0_newTarget_variance.csv')
    newx_observed_df_gp0 = pd.read_csv(model_input.output_folder+'gp0_newRecommendation.csv')
    index_observed_df_gp0 = pd.read_csv(model_input.output_folder+'gp0_IndexRecommendation.csv')
    
if model_input.GP_L_BO:
    best_observed_df_gpL = pd.read_csv(model_input.output_folder+'gpL_best.csv')
    newy_observed_df_gpL = pd.read_csv(model_input.output_folder+'gpL_newTarget.csv')
    newy_var_observed_df_gpL = pd.read_csv(model_input.output_folder+'gpL_newTarget_variance.csv')
    newx_observed_df_gpL = pd.read_csv(model_input.output_folder+'gpL_newRecommendation.csv')
    index_observed_df_gpL = pd.read_csv(model_input.output_folder+'gpL_IndexRecommendation.csv')
    
if model_input.GP_NN_BO:
    best_observed_df_gpNN = pd.read_csv(model_input.output_folder+'gpNN_best.csv')
    newy_observed_df_gpNN = pd.read_csv(model_input.output_folder+'gpNN_newTarget.csv')
    newy_var_observed_df_gpNN = pd.read_csv(model_input.output_folder+'gpNN_newTarget_variance.csv')
    newx_observed_df_gpNN = pd.read_csv(model_input.output_folder+'gpNN_newRecommendation.csv')
    index_observed_df_gpNN = pd.read_csv(model_input.output_folder+'gpNN_IndexRecommendation.csv')
    

Reading data for the input dataset type:  MPEA


In [3]:
comp = XX.loc[index_observed_df_gpL['gpL_trial1']].reset_index()
target = newy_observed_df_gpL[:]
comp['Hardness Value'] = target.mean(axis=1)
comp = comp.sort_values(by=['Hardness Value'], ascending=False)

In [4]:
comp.to_csv(model_input.output_folder+'Recommendations.csv',index=False)

In [ ]:
Round1_reco = pd.read_csv(model_input.output_folder+'../mpea_hv_forEddie_0.01p_FirstPass_Aug/Recommendations.csv')

In [ ]:
comp_shortlist = comp[:][0:20]
Round1_reco_shortlist = Round1_reco[:][0:20]